In [4]:
import torch
from ultralytics import YOLO
import cv2
import time
import os

# --- Global Configuration ---
ENABLE_VISUALIZATION = False # Set to False to disable visualization and see pure inference timea

# --- Paths ---
model_path = "yolov8n_lidar.pt"
image_path = "frame_0_0_0_0.png"
output_path = "frame_0_0_0_0-detected.png"

# --- Load YOLO model ---
model = YOLO(model_path, task='detect')
model_type = os.path.splitext(model_path)[-1].lstrip('.').upper()
print(f"Running model type: {model_type}")

# --- Warmup run (first inference is always slower) ---
print("Warming up model...")
_ = model(image_path, imgsz=[64, 384])

# --- Pure Inference (just the model) ---
print("Running inference test...")
start_time = time.time()
results = model(image_path, imgsz=[64, 384])
pure_inference_time = time.time() - start_time
print(f"Pure inference time: {pure_inference_time*1000:.1f} ms")

# --- Full Processing (including visualization) ---
start_time = time.time()

# Load image and process detections
if ENABLE_VISUALIZATION:
    im_bgr = cv2.imread(image_path)

# Draw detections
for r in results:
    boxes_xyxy = r.boxes.xyxy.cpu().numpy()
    boxes_xywh = r.boxes.xywh.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    class_ids = r.boxes.cls.cpu().numpy().astype(int)
    names = r.names

    for (x1, y1, x2, y2), (cx, cy, w, h), conf, class_id in zip(boxes_xyxy, boxes_xywh, confs, class_ids):
        label = f"{names[class_id]} {conf:.2f}"
        print(f"[{label}] -> center: ({cx:.1f}, {cy:.1f}), size: ({w:.1f}x{h:.1f})")

        if ENABLE_VISUALIZATION:
            color = (0, 255, 0)
            cv2.rectangle(im_bgr, (int(x1), int(y1)), (int(x2), int(y2)), color, 1)
            cv2.putText(im_bgr, label, (int(x1), max(int(y1)-2, 0)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.25, color, 1, cv2.LINE_AA)

# Show and save (only if visualization is enabled)
if ENABLE_VISUALIZATION:
    cv2.imshow("YOLOv8 Detection", im_bgr)
    cv2.imwrite(output_path, im_bgr)
    # cv2.waitKey(0)  # Removed - this waits for user input and affects timing
    cv2.destroyAllWindows()
    print("Visualization enabled - image displayed and saved")
else:
    print("Visualization disabled - pure inference only")

full_processing_time = time.time() - start_time
print(f"Full processing time: {full_processing_time:.4f} seconds")
print(f"Visualization overhead: {full_processing_time:.4f} seconds")
print(f"Total time (inference + processing): {pure_inference_time + full_processing_time:.4f} seconds")

Running model type: PT
Warming up model...

image 1/1 /home/soheil/frame_0_0_0_0.png: 64x384 1 chair, 1 box, 1 desk, 1 door_frame, 43.0ms
Speed: 0.4ms preprocess, 43.0ms inference, 1.3ms postprocess per image at shape (1, 3, 64, 384)
Running inference test...

image 1/1 /home/soheil/frame_0_0_0_0.png: 64x384 1 chair, 1 box, 1 desk, 1 door_frame, 41.3ms
Speed: 0.4ms preprocess, 41.3ms inference, 1.3ms postprocess per image at shape (1, 3, 64, 384)
Pure inference time: 47.8 ms
[box 0.94] -> center: (242.5, 22.7), size: (60.7x29.3)
[desk 0.89] -> center: (64.2, 19.1), size: (120.5x38.1)
[chair 0.86] -> center: (339.6, 21.8), size: (57.6x27.5)
[door_frame 0.84] -> center: (196.3, 33.0), size: (44.9x19.3)
Visualization disabled - pure inference only
Full processing time: 0.0016 seconds
Visualization overhead: 0.0016 seconds
Total time (inference + processing): 0.0494 seconds
